In [ ]:
#install the packages
!pip install sentence-transformers
!pip install umap-learn
# !pip install nltk
# !pip install torch
# !pip install tensorflow
# !pip install tensorboard

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector
#load tensorboards with magics
%tensorflow_version 2.x
%load_ext tensorboard

In [ ]:
#import required packages
import random
from tqdm import tqdm
import numpy as np
import os
from sentence_transformers import SentenceTransformer
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

In [ ]:
!git clone https://github.com/akashjorss/sentence_bert
!unzip sentence_bert/sentences.zip
import pickle
with open('sentences.bin', 'rb') as f:
  sentences = pickle.load(f)

In [ ]:
data = random.choices(sentences,k = 100000)
# data = [s.lower() for s in data]

# stop = stopwords.words('english')
# sentiment_tags = ['ADJ', 'ADV', 'RB', 'VBZ']

# for i in tqdm(range(len(data))):
#   s = data[i]
#   words = word_tokenize(s)
#   #remove stop words
#   words = [w for w in words if w not in stop]
#   #remove sentiment words
#   pos_tags = nltk.pos_tag(words)
#   words = [t[0] for t in pos_tags if t[1] not in sentiment_tags]
#   s = " ".join(words)
#   data[i] = s

In [ ]:
#make sentence vectors
model = SentenceTransformer('distilbert-base-nli-mean-tokens') #is it the correct sentence-bert?
# model.to('cuda') -> not needed
embeddings = model.encode(data, show_progress_bar=True)

In [ ]:
#dim reduction using umap
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=3, 
                            metric='cosine').fit_transform(embeddings)

In [ ]:
#visualize high dim data in embedding projector
# Set up a logs directory, so Tensorboard knows where to look for files
import numpy as np
log_dir='./logs/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save the weights we want to analyse as a variable. 
umap_weights = tf.Variable(random.choices(umap_embeddings, k=10000))
# Create a checkpoint from embedding, the filename and key are
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=umap_weights)
checkpoint.save(os.path.join(log_dir, "umap_embedding.ckpt"))

# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
# embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
%tensorboard --logdir ./logs/

In [ ]:
#perform dim reduction hdbscan and get the topics
#Do we need to reduce the dimensionality? Can't clustering work in high dim data?
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

In [ ]:
!pip install hdbscan
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=20,
                          metric='euclidean',  #can use diff method here                    
                          cluster_selection_method='eom').fit(umap_embeddings) 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import umap
# Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# pca_data = PCA(n_components=2).fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()

docs_df = pd.DataFrame(data, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

top_n_words

# from sklearn.metrics.pairwise import cosine_similarity
# for i in range(20):
#     # Calculate cosine similarity
#     similarities = cosine_similarity(tf_idf.T)
#     np.fill_diagonal(similarities, 0)

#     # Extract label to merge into and from where
#     topic_sizes = docs_df.groupby(['Topic']).count().sort_values("Doc", ascending=False).reset_index()
#     topic_to_merge = topic_sizes.iloc[-1].Topic
#     topic_to_merge_into = np.argmax(similarities[topic_to_merge + 1]) - 1

#     # Adjust topics
#     docs_df.loc[docs_df.Topic == topic_to_merge, "Topic"] = topic_to_merge_into
#     old_topics = docs_df.sort_values("Topic").Topic.unique()
#     map_topics = {old_topic: index - 1 for index, old_topic in enumerate(old_topics)}
#     docs_df.Topic = docs_df.Topic.map(map_topics)
#     docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

#     # Calculate new topic words
#     m = len(data)
#     tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m)
#     top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)

# topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)
